In [1]:
import numpy as np
np.random.seed(37) # 使得每次运行得到的随机数都一样

In [2]:
# 1, 准备数据集
from nltk.corpus import movie_reviews
pos_fileIds=movie_reviews.fileids('pos') # 加载积极文本文件
neg_fileIds=movie_reviews.fileids('neg') # 消极文本文件

print(len(pos_fileIds)) # 1000
print(len(neg_fileIds)) # 1000 

print(pos_fileIds[:5])
print(neg_fileIds[:5])

# 由此可看出，movie_reviews.fileids是加载各种类别文本的文件，
# 并返回该文件名组成的list

# 如果想要查看某个文本文件的内容，可以使用
print(movie_reviews.words(fileids=['pos/cv000_29590.txt']))

1000
1000
['pos/cv000_29590.txt', 'pos/cv001_18431.txt', 'pos/cv002_15918.txt', 'pos/cv003_11664.txt', 'pos/cv004_11636.txt']
['neg/cv000_29416.txt', 'neg/cv001_19502.txt', 'neg/cv002_17424.txt', 'neg/cv003_12683.txt', 'neg/cv004_12641.txt']
['films', 'adapted', 'from', 'comic', 'books', 'have', ...]


In [3]:
# 2, 处理数据集
def extract_features(word_list):
    '''专门一个函数来提取特征'''
    return dict([(word,True) for word in word_list]) # 此处加True的作用是构成dict,实质意义不大

pos_features=[(extract_features(movie_reviews.words(fileids=[f])),'Pos') 
              for f in pos_fileIds]
neg_features=[(extract_features(movie_reviews.words(fileids=[f])),'Neg') 
              for f in neg_fileIds]
print(pos_features[:3]) # 打印下看看内容是否正确

dataset=pos_features+neg_features # 将两部分结合起来作为一个dataset

[({'films': True, 'adapted': True, 'from': True, 'comic': True, 'books': True, 'have': True, 'had': True, 'plenty': True, 'of': True, 'success': True, ',': True, 'whether': True, 'they': True, "'": True, 're': True, 'about': True, 'superheroes': True, '(': True, 'batman': True, 'superman': True, 'spawn': True, ')': True, 'or': True, 'geared': True, 'toward': True, 'kids': True, 'casper': True, 'the': True, 'arthouse': True, 'crowd': True, 'ghost': True, 'world': True, 'but': True, 'there': True, 's': True, 'never': True, 'really': True, 'been': True, 'a': True, 'book': True, 'like': True, 'hell': True, 'before': True, '.': True, 'for': True, 'starters': True, 'it': True, 'was': True, 'created': True, 'by': True, 'alan': True, 'moore': True, 'and': True, 'eddie': True, 'campbell': True, 'who': True, 'brought': True, 'medium': True, 'to': True, 'whole': True, 'new': True, 'level': True, 'in': True, 'mid': True, '80s': True, 'with': True, '12': True, '-': True, 'part': True, 'series': Tru

In [4]:
# 构建模型，训练模型
from nltk import NaiveBayesClassifier
from nltk.classify import accuracy as nltk_accuracy

np.random.shuffle(dataset)
rows=int(len(dataset)*0.8) # 80%为train set
train_set,test_set=dataset[:rows],dataset[rows:]
print('Num of train_set: ',len(train_set),
      '\nNum of test_set: ',len(test_set))
clf=NaiveBayesClassifier.train(train_set)

# 查看该模型在test set上的表现
acc=nltk_accuracy(clf,test_set)
print('Accuracy: {:.2f}%'.format(acc*100))

Num of train_set:  1600 
Num of test_set:  400
Accuracy: 69.25%


In [5]:
# 查看模型内部信息
# 该分类器是分析某段文本中哪些单词与“积极”的关联最大，
# 哪些与“消极”的关联最大，进而分析这些关键词的出现来判断某句话是积极或消极

# 打印这些关键词
for key_word in clf.most_informative_features()[:10]:
    print(key_word[0])

insulting
astounding
3000
outstanding
offbeat
palpable
wonderfully
stupidity
hatred
slip


In [6]:
# 用该模型来预测新样本，查看新句子的情感是积极还是消极
new_samples = [
        "It is an amazing movie", 
        "This is a dull movie. I would never recommend it to anyone.",
        "The cinematography is pretty great in this movie", 
        "The direction was terrible and the story was all over the place" 
    ]

for sample in new_samples:
    predict_P=clf.prob_classify(extract_features(sample.split()))
    pred_sentiment=predict_P.max()
    print('Sample: {}, Type: {}, Probability: {:.2f}%'.format(
        sample,pred_sentiment,predict_P.prob(pred_sentiment)*100))


Sample: It is an amazing movie, Type: Pos, Probability: 58.08%
Sample: This is a dull movie. I would never recommend it to anyone., Type: Neg, Probability: 77.31%
Sample: The cinematography is pretty great in this movie, Type: Pos, Probability: 64.29%
Sample: The direction was terrible and the story was all over the place, Type: Neg, Probability: 65.96%
